In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit.random import random_circuit

def create_random_circuit(num_qubits, depth):
    # Qiskit's built-in random_circuit function: create a random quantum circuit of given number of qubits and depth.
         # measure=True ensures that all qubits are measured at the end of the circuit
    qc = random_circuit(num_qubits, depth, measure=True)
    return qc

num_qubits = 3
depth = 5
my_random_qc = create_random_circuit(num_qubits, depth)
print("Generated a random circuit:")
print(my_random_qc)


Generated a random circuit:
          ┌───┐     ┌────────────┐ ┌──────────────────┐┌────────────┐┌───┐ ┌─┐»
q_0: ─────┤ T ├─────┤ Ry(1.3364) ├─┤ R(5.9896,1.9378) ├┤ Ry(6.2757) ├┤ Y ├─┤M├»
     ┌────┴───┴────┐└───┬───┬────┘┌┴──────────────────┤└───┬───┬────┘├───┴┐└╥┘»
q_1: ┤ Rz(0.60924) ├────┤ S ├─────┤ U2(1.3378,3.6439) ├────┤ H ├─────┤ √X ├─╫─»
     └──────┬──────┘    ├───┤     └───┬────────────┬──┘    └─┬─┘     ├────┤ ║ »
q_2: ───────■───────────┤ I ├─────────┤ U1(4.3487) ├─────────■───────┤ √X ├─╫─»
                        └───┘         └────────────┘                 └────┘ ║ »
c: 3/═══════════════════════════════════════════════════════════════════════╩═»
                                                                            0 »
«           
«q_0: ──────
«     ┌─┐   
«q_1: ┤M├───
«     └╥┘┌─┐
«q_2: ─╫─┤M├
«      ║ └╥┘
«c: 3/═╩══╩═
«      1  2 


In [2]:
# need two quantum computers: one is noisy one is ideal:

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error

# no noise model
ideal_simulator = AerSimulator()

# noise
# depolarizing error simulates a random Pauli error (X, Y, or Z) occurring with a certain probability
p_error = 0.01  # prob of a single-qubit gate error

error_1 = depolarizing_error(p_error, 1)
error_2 = depolarizing_error(p_error, 2) 

noise_model = NoiseModel()


noise_model.add_all_qubit_quantum_error(error_1, ['sx', 'rz', 'id'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

noisy_simulator = AerSimulator(noise_model=noise_model)

print("Simulators are ready.")


Simulators are ready.


In [6]:
from qiskit import transpile

# run the circuit:

qc = my_random_qc # random circuit from cell 1
shots = 1024 

# Transpile the circuit for the AerSimulator basis gates
basis_gates = ['cx', 'id', 'rz', 'sx', 'x']
qc_ideal = transpile(qc, basis_gates=basis_gates)
qc_noisy = transpile(qc, basis_gates=basis_gates)

ideal_job = ideal_simulator.run(qc_ideal, shots=shots)
ideal_result = ideal_job.result()
ideal_counts = ideal_result.get_counts()

noisy_job = noisy_simulator.run(qc_noisy, shots=shots)
noisy_result = noisy_job.result()
noisy_counts = noisy_result.get_counts()

print("\n--- Simulation Results ---")
print("Ideal Counts:", ideal_counts)
print("Noisy Counts:", noisy_counts)


--- Simulation Results ---
Ideal Counts: {'000': 115, '101': 379, '100': 112, '001': 403, '111': 9, '011': 5, '110': 1}
Noisy Counts: {'001': 355, '101': 348, '111': 35, '100': 115, '000': 123, '011': 26, '110': 16, '010': 6}


In [ ]:
# now, we must convert counts to probabilities:
import numpy as np
def counts_to_vector(counts, num_qubits):
    # goal: converts a qiskit counts dictionary to a fixed length prob vector
    num_outcomes = 2**num_qubits # dimension
    prob_vector = np.zeros(num_outcomes)
    total_shots = sum(counts.values())

    for bitstring, count in counts.items():
        # convert bitstring to an integer index (e.g. '011' -> 3)
        index = int(bitstring, 2)
        prob_vector[index] = count / total_shots
    return prob_vector

In [ ]:
# example
ideal_vector = counts_to_vector(ideal_counts, num_qubits)
noisy_vector = counts_to_vector(noisy_counts, num_qubits)

print("\n--- Processed Vectors ---")
print("Ideal Vector (y):", ideal_vector)
print("Noisy Vector (X):", noisy_vector)
print("Vector Length:", len(ideal_vector))
print("Sum of probabilities:", np.sum(ideal_vector))



--- Processed Vectors ---
Ideal Vector (y): [0.00097656 0.25976562 0.24609375 0.00097656 0.00097656 0.24023438
 0.25       0.00097656]
Noisy Vector (X): [0.00683594 0.24707031 0.22558594 0.00292969 0.01074219 0.25097656
 0.25390625 0.00195312]
Vector Length: 8
Sum of probabilities: 1.0
